# Python Concurrency and Networking
This notebook covers threading, multiprocessing, async/await, sockets, and HTTP requests with real-life use cases, best practices, and code examples.

## 1. Threading
**Definition:** Threading allows concurrent execution of code, useful for I/O-bound tasks.

### Importing Threading and Time Modules

**Introduction:**
To use threading in Python, you need to import the `threading` and `time` modules.

**Real-life use case:**
These modules are used for running concurrent tasks and simulating delays in code.

**What the code does:**
The next code cell imports the required modules for threading examples.

In [ ]:
import threading
import time

### Basic Threading Example

**Introduction:**
A thread allows you to run a function concurrently with the main program.

**Real-life use case:**
Downloading files in the background while the main program continues.

**What the code does:**
The next code cell defines a function that prints numbers with a delay, and shows how to run it in a separate thread.

In [ ]:
def print_numbers():
    """Print numbers with a delay to simulate work."""
    for i in range(3):
        print(f'Thread: {i}')
        time.sleep(1)

# Create and start a thread
thread = threading.Thread(target=print_numbers)
thread.start()
print("Main thread continues while child thread runs")
thread.join()
print("Thread completed")

### Running Multiple Threads in Parallel

**Introduction:**
You can start several threads to perform tasks in parallel, which is useful for handling multiple I/O operations at once.

**Real-life use case:**
Processing multiple files or network requests simultaneously.

**What the code does:**
The next code cell defines a worker function and starts multiple threads to run it.

In [ ]:
def worker(name):
    """Worker function that reports its name and timestamps."""
    print(f'Worker {name} starting')
    time.sleep(2)
    print(f'Worker {name} finished')

threads = []
for i in range(3):
    t = threading.Thread(target=worker, args=(f'#{i}',))
    threads.append(t)
    t.start()
for t in threads:
    t.join()
print("All workers completed")

### Running a Background Task with a Daemon Thread

**Introduction:**
Daemon threads run in the background and automatically exit when the main program finishes.

**Real-life use case:**
Background monitoring or periodic tasks that should not block program exit.

**What the code does:**
The next code cell creates a daemon thread for a background task while the main thread continues its work.

In [ ]:
def background_task():
    for i in range(5):
        print(f"Background task: step {i}")
        time.sleep(0.5)

background_thread = threading.Thread(target=background_task)
background_thread.daemon = True
background_thread.start()
for i in range(3):
    print(f"Main thread: step {i}")
    time.sleep(0.7)
print("Main thread finished (background thread may not have completed)")

### Thread Synchronization with Lock

**Introduction:**
When multiple threads access shared data, you need synchronization to prevent race conditions.

**Real-life use case:**
Safely updating a shared counter from multiple threads.

**What the code does:**
The next code cell demonstrates using a Lock to synchronize access to a shared variable.

In [ ]:
counter = 0
count_lock = threading.Lock()

def increment_counter():
    global counter
    for _ in range(100000):
        with count_lock:
            counter += 1
thread1 = threading.Thread(target=increment_counter)
thread2 = threading.Thread(target=increment_counter)
thread1.start()
thread2.start()
thread1.join()
thread2.join()
print(f"Final counter value with locks: {counter}")

### Producer-Consumer Pattern with Queue

**Introduction:**
The producer-consumer pattern uses a queue to safely pass data between threads.

**Real-life use case:**
A web crawler where one thread fetches URLs (producer) and another processes the content (consumer).

**What the code does:**
The next code cell demonstrates a producer thread putting items in a queue and a consumer thread processing them.

In [ ]:
import queue

task_queue = queue.Queue()

def producer():
    for i in range(5):
        item = f"Item-{i}"
        task_queue.put(item)
        print(f"Produced {item}")
        time.sleep(0.5)
    task_queue.put(None)

def consumer():
    while True:
        item = task_queue.get()
        if item is None:
            break
        print(f"Consumed {item}")
        task_queue.task_done()
        time.sleep(1)
prod_thread = threading.Thread(target=producer)
cons_thread = threading.Thread(target=consumer)
prod_thread.start()
cons_thread.start()
prod_thread.join()
cons_thread.join()
print("Producer-Consumer demonstration completed")

### Note on Python's Global Interpreter Lock (GIL)

**Introduction:**
The Global Interpreter Lock (GIL) affects how threads execute Python code.

**Real-life use case:**
Understanding the GIL helps you choose between threading and multiprocessing for your application.

**What the code does:**
The next code cell explains the GIL and when to use threading vs. multiprocessing.

In [ ]:
print("The GIL prevents multiple native threads from executing Python bytecode at once.")
print("This means threading is most useful for I/O-bound tasks rather than CPU-bound tasks.")
print("For CPU-bound tasks, consider multiprocessing instead.")

## 2. Multiprocessing
**Definition:** Multiprocessing runs code in separate processes, ideal for CPU-bound tasks.

### Importing Multiprocessing and Related Modules

**Introduction:**
To use multiprocessing in Python, you need to import the `multiprocessing` module and others as needed.

**Real-life use case:**
Multiprocessing is used for parallelizing CPU-intensive tasks, such as image processing or scientific computations.

**What the code does:**
The next code cell imports the required modules for multiprocessing examples.

In [ ]:
import multiprocessing
import time
import os
import numpy as np

### Basic Multiprocessing Pool Example

**Introduction:**
A process pool allows you to run a function in parallel across multiple processes.

**Real-life use case:**
Speeding up computations by distributing work across CPU cores.

**What the code does:**
The next code cell demonstrates using a pool to compute squares in parallel.

In [ ]:
def square(x):
    """Simple CPU-bound function that squares a number."""
    process_id = os.getpid()
    print(f"Process {process_id}: Computing square of {x}")
    time.sleep(0.5)
    return x * x

if __name__ == '__main__':
    with multiprocessing.Pool(processes=2) as pool:
        results = pool.map(square, [1, 2, 3, 4])
        print(f"Results: {results}")

### Using the Process Class Directly

**Introduction:**
You can create and manage individual processes for more control.

**Real-life use case:**
Running independent tasks that do not need to share state.

**What the code does:**
The next code cell shows how to start and join multiple processes.

In [ ]:
def worker_function(name):
    process_id = os.getpid()
    print(f"Worker {name} (PID: {process_id}) starting")
    time.sleep(1)
    print(f"Worker {name} (PID: {process_id}) finished")

if __name__ == '__main__':
    processes = []
    for i in range(3):
        p = multiprocessing.Process(target=worker_function, args=(f'Process-{i}',))
        processes.append(p)
        p.start()
    for p in processes:
        p.join()
    print("All worker processes completed")

### CPU-bound Task Example: Finding Prime Numbers

**Introduction:**
Multiprocessing is ideal for CPU-bound tasks like finding prime numbers in a range.

**Real-life use case:**
Parallelizing mathematical computations to utilize all CPU cores.

**What the code does:**
The next code cell splits a range into chunks and counts primes in parallel.

In [ ]:
def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def count_primes_in_range(start, end):
    return sum(1 for i in range(start, end) if is_prime(i))

if __name__ == '__main__':
    range_start = 1
    range_end = 100000
    num_processes = multiprocessing.cpu_count()
    chunk_size = (range_end - range_start) // num_processes
    chunks = [(range_start + i * chunk_size, range_start + (i + 1) * chunk_size) for i in range(num_processes)]
    start_time = time.time()
    with multiprocessing.Pool(processes=num_processes) as pool:
        results = pool.starmap(count_primes_in_range, chunks)
    total_primes = sum(results)
    mp_time = time.time() - start_time
    print(f"Found {total_primes} prime numbers in range {range_start}-{range_end}")
    print(f"Multiprocessing time: {mp_time:.2f} seconds")

### Shared Memory Example with multiprocessing.Value

**Introduction:**
Processes do not share memory by default, but you can use shared values for communication.

**Real-life use case:**
Safely updating a shared counter from multiple processes.

**What the code does:**
The next code cell demonstrates using a shared value and a lock.

In [ ]:
if __name__ == '__main__':
    counter = multiprocessing.Value('i', 0)
    def increment_counter(count, lock):
        for _ in range(count):
            with lock:
                counter.value += 1
    lock = multiprocessing.Lock()
    processes = []
    num_increments = 1000
    for _ in range(4):
        p = multiprocessing.Process(target=increment_counter, args=(num_increments, lock))
        processes.append(p)
        p.start()
    for p in processes:
        p.join()
    print(f"Final counter value: {counter.value}")

### Data Processing with Multiprocessing and NumPy

**Introduction:**
Multiprocessing can be used to process large datasets in parallel.

**Real-life use case:**
Speeding up data analysis or machine learning preprocessing.

**What the code does:**
The next code cell splits a NumPy array into chunks and processes them in parallel.

In [ ]:
def process_chunk(chunk):
    result = np.mean(chunk) * np.std(chunk)
    time.sleep(0.2)
    return result

if __name__ == '__main__':
    data = np.random.rand(1000000)
    chunks = np.array_split(data, 4)
    start_time = time.time()
    with multiprocessing.Pool(processes=4) as pool:
        results = pool.map(process_chunk, chunks)
    combined_result = np.mean(results)
    mp_time = time.time() - start_time
    print(f"Parallel processing result: {combined_result:.6f}")
    print(f"Processing time: {mp_time:.2f} seconds")

### Tips for Effective Multiprocessing

**Introduction:**
Following best practices ensures efficient and safe use of multiprocessing.

**Real-life use case:**
Avoiding common pitfalls like race conditions and excessive data transfer.

**What the code does:**
The next code cell lists tips for using multiprocessing effectively.

In [ ]:
print("1. Always protect shared memory with locks")
print("2. Minimize data transfer between processes")
print("3. Use multiprocessing for CPU-bound tasks, not I/O-bound tasks")
print("4. Use 'if __name__ == __main__' guard for Windows compatibility")
print("5. Consider process pools for managing worker processes")
print("6. Be aware of overhead - only use multiprocessing when tasks are substantial")